In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 26.2 MB/s eta 0:00:00


In [4]:
import pandas as pd
import re
import konlpy
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import numpy as np
from tqdm import tqdm
import urllib.request

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-2, 2))

###강의자료
ratings_train.txt 파일을 탭(tab) 구분자로 읽어서 DataFrame으로 저장합니다.
전체 데이터 중 처음 10,000개 행(row)만 선택합니다.
원본 데이터에서 'id' 열과 'document' 열만 선택합니다.
나머지 열(예: 'label' 같은 것들)은 제거됩니다.
열 이름을 변경합니다.
여기서는 'document' 열의 이름을 'sentence'로 바꿔서 문장을 담는 열임을 명확하게 합니다.

###과제
my_senti.txt를 tab 구분자로 읽어서 dataframe으로 불러오고, 열 이름을 sentence로 지정. id 열을 추가하기 위해 현재 index를 'id'열로 지정.

In [15]:
df = pd.read_table('./sample_data/my_senti.txt', header=None, names=['sentence'])
df['id'] = df.index
df

,sentence,id
0,감기에 걸려서 너무 피곤하고 아팠다.,0
1,할 것들이 많아서 스트레스를 받았다.,1
2,주말을 너무 게으르게 보냈다. 작은거 하나라도 완성하는 의미있는 시간을 보내자!,2
3,이해가 되지 않지만 그럴수도 있다고 생각해보자.,3
4,추가 근무가 들어와서 짜증이 났지만 도움이 될 수 있어서 좋다.,4
5,서두르지 말고 힘을 빼자. 지치지 말고 바라보는 시간도 가져야 한다.,5
6,많이 지쳐서 잠깐만 쉴게요. 마음 속 상처를 돌봐줘야 해요.,6
7,여러분들이 웃는 모습을 볼 수 있어서 좋아요. 여러분이 웃을 수 있다면 기꺼이 할 ...,7
8,여전히 찝찝하고 편하지 않다. 누구에게 권할 자신이 없다.,8
9,누군가를 사랑하는 것은 어려운 것 같아요.,9


In [16]:
sent_dic = pd.read_csv('./sample_data/SentiWord_Dict.txt',sep = '\t',header=None)
# sent_dic.iloc[14850,0]='갈등'

pos_dic = sent_dic[sent_dic[1]>0]
neg_dic = sent_dic[sent_dic[1]<0]
neu_dic = sent_dic[sent_dic[1]==0]

In [17]:
okt = konlpy.tag.Okt()

def text_preprocess(x):
    text=[]
    a = re.sub('[^가-힣0-9a-zA-Z\\s]', '',x)
    for j in a.split():
        text.append(j)
    return ' '.join(text)

def tokenize(x):
    text = []
    tokens = okt.pos(x)
    for token in tokens :
        if token[1] == 'Adjective' or token[1]=='Adverb' or token[1] == 'Determiner' or token[1] == 'Noun' or token[1] == 'Verb' or 'Unknown':
            text.append(token[0])
    return text

In [18]:
tqdm.pandas()
df['comment'] = df['sentence'].apply(lambda x : text_preprocess(x))
df['comment'] = df['comment'].progress_apply(lambda x: tokenize(x))
df

100%|██████████| 20/20 [00:00<00:00, 40.10it/s]


,sentence,id,comment
0,감기에 걸려서 너무 피곤하고 아팠다.,0,"[감기, 에, 걸려서, 너무, 피곤하고, 아팠다]"
1,할 것들이 많아서 스트레스를 받았다.,1,"[할, 것, 들, 이, 많아서, 스트레스, 를, 받았다]"
2,주말을 너무 게으르게 보냈다. 작은거 하나라도 완성하는 의미있는 시간을 보내자!,2,"[주말, 을, 너무, 게으르게, 보냈다, 작은거, 하나, 라도, 완성, 하는, 의미..."
3,이해가 되지 않지만 그럴수도 있다고 생각해보자.,3,"[이해, 가, 되지, 않지만, 그럴수도, 있다고, 생각, 해보자]"
4,추가 근무가 들어와서 짜증이 났지만 도움이 될 수 있어서 좋다.,4,"[추가, 근무, 가, 들어와서, 짜증, 이, 났지만, 도움, 이, 될, 수, 있어서..."
5,서두르지 말고 힘을 빼자. 지치지 말고 바라보는 시간도 가져야 한다.,5,"[서두르지, 말고, 힘, 을, 빼자, 지치지, 말고, 바라보는, 시간, 도, 가져야..."
6,많이 지쳐서 잠깐만 쉴게요. 마음 속 상처를 돌봐줘야 해요.,6,"[많이, 지쳐서, 잠깐, 만, 쉴게요, 마음, 속, 상처, 를, 돌봐줘야, 해, 요]"
7,여러분들이 웃는 모습을 볼 수 있어서 좋아요. 여러분이 웃을 수 있다면 기꺼이 할 ...,7,"[여러분, 들, 이, 웃는, 모습, 을, 볼, 수, 있어서, 좋아요, 여러분, 이,..."
8,여전히 찝찝하고 편하지 않다. 누구에게 권할 자신이 없다.,8,"[여전히, 찝찝하고, 편하지, 않다, 누구, 에게, 권할, 자신, 이, 없다]"
9,누군가를 사랑하는 것은 어려운 것 같아요.,9,"[누군가, 를, 사랑, 하는, 것, 은, 어려운, 것, 같아요]"


In [19]:
def make_sent_dict(x) :
    pos=[]
    neg=[]
    tmp={}

    for sentence in tqdm(x):
        for word in sentence :
            target = sent_dic[sent_dic[0]==word]
            if len(target)==1: # 기존에 있는 단어라면 그대로 사용
                score = float(target[1])
                if score > 0:
                    pos.append(word)
                elif score < 0:
                    neg.append(word)
            tmp[word] = {'W':0,'WP':0,'WN':0} # 감성사전 구성
    pos = list(set(pos))
    neg = list(set(neg))

    for sentence in tqdm(x):
        for word in sentence :
            tmp[word]['W'] += 1 # 빈도 수
            for po in pos :
                if po in sentence:
                    tmp[word]['WP'] += 1 # 긍정단어과 같은 문장 내 단어일 때
                    break
            for ne in neg:
                if ne in sentence:
                    tmp[word]['WN'] += 1 # 부정단어와 같은 문장내 단어일 때
                    break
    return pos, neg, pd.DataFrame(tmp)

In [20]:
pos, neg, new_dict = make_sent_dict(df['comment'].values)
new_dict

  0%|          | 0/20 [00:00<?, ?it/s]<ipython-input-19-eb8405f5292c>:10: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  score = float(target[1])
100%|██████████| 20/20 [00:00<00:00, 13985.68it/s]


,감기,에,걸려서,너무,피곤하고,아팠다,할,것,들,이,...,어울리지,않는다,활짝,얼굴,뒤,슬픔,보인다,웃지,않아도,괜찮아
W,1,4,1,2,1,1,3,3,2,11,...,1,1,1,1,1,1,1,1,1,1
WP,0,2,0,1,0,0,2,2,1,6,...,1,1,1,1,1,1,1,1,1,1
WN,1,2,1,2,1,1,1,3,1,6,...,0,0,1,1,1,1,1,1,1,1


In [21]:
pos

['사랑', '완성', '좋다', '행복하다', '도움', '웃는', '여유']

In [22]:
neg

['슬프다',
 '두렵고',
 '감기',
 '상처',
 '해',
 '지쳐서',
 '짜증',
 '게으르게',
 '슬픔',
 '어려운',
 '없다',
 '스트레스',
 '어둡고']

In [23]:
def make_score_dict(d,p,n):
    N=sum(d.iloc[0,::])
    pos_cnt=sum(d.loc[::,p].iloc[0,::])
    neg_cnt=sum(d.loc[::,n].iloc[0,::])

    trans =d.T
    trans['neg_cnt']=neg_cnt
    trans['pos_cnt']=pos_cnt
    trans['N']=N

    trans['MI_P']=np.log2(trans['WP']*trans['N']/trans['W']*trans['pos_cnt'])
    trans['MI_N']=np.log2(trans['WN']*trans['N']/trans['W']*trans['neg_cnt'])
    trans['SO_MI']=trans['MI_P'] - trans['MI_N']

    trans = trans.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
    trans = trans.sort_values(by=['SO_MI'],ascending=False)
    return trans

In [24]:
t_dict = make_score_dict(new_dict,pos,neg)
t_dict['SO_MI'] = scaler.fit_transform(t_dict['SO_MI'].values.reshape(-1,1))
t_dict

/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)


,W,WP,WN,neg_cnt,pos_cnt,N,MI_P,MI_N,SO_MI
수,6,6,1,13,12,212,11.312883,8.843398,2.000000
있어서,3,3,1,13,12,212,11.312883,9.843398,0.884228
웃는,3,3,1,13,12,212,11.312883,9.843398,0.884228
시간,3,2,1,13,12,212,10.727920,9.843398,0.231544
할,3,2,1,13,12,212,10.727920,9.843398,0.231544
도,3,2,1,13,12,212,10.727920,9.843398,0.231544
있는,2,2,1,13,12,212,11.312883,10.428360,0.231544
을,7,6,3,13,12,212,11.090491,10.205968,0.231544
좋다,4,4,2,13,12,212,11.312883,10.428360,0.231544
의미,1,1,1,13,12,212,11.312883,11.428360,-0.884228


In [25]:
def update_dict(d):
    add_Dic = {0:[],1:[]}
    for i in d.T.items():
        if i[0] not in list(sent_dic[0]):
            if len(i[0]) > 1:
                add_Dic[0].append(i[0])
                add_Dic[1].append(i[1]['SO_MI'])

    add_Dic=pd.DataFrame(add_Dic)
    Sentiment=pd.merge(sent_dic,add_Dic,'outer')
    return Sentiment

In [26]:
add_dict =update_dict(t_dict)
add_dict

<ipython-input-25-9c96c53f53b2>:10: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  Sentiment=pd.merge(sent_dic,add_Dic,'outer')


,0,1
0,(-;,1.0
1,(-_-),-1.0
2,(;_;),-1.0
3,(T_T),-1.0
4,(^-^),1.0
...,...,...
14867,힘차고 대단한,2.0
14868,힘차고 튼튼하다,2.0
14869,힘차다,2.0
14870,힘찬,2.0


In [27]:
def get_cnt(x):
    cnt = 0
    for word in list(set(x)):
        target = add_dict[add_dict[0]==word]
        if len(target)==1:
            cnt += float(target[1])
    return cnt

def get_ratio(x):
    score = x['score']
    length = np.log10(len(x['comment']))+1
    try:
        ratio= round(score/length,2)
    except:
        ratio = 0
    return ratio

In [28]:
tqdm.pandas()
df['score']= df['comment'].progress_apply(lambda x : get_cnt(x))
df['ratio'] = df.apply(lambda x: get_ratio(x), axis = 1)

  0%|          | 0/20 [00:00<?, ?it/s]<ipython-input-27-1fcf478c7d24>:6: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  cnt += float(target[1])
100%|██████████| 20/20 [00:00<00:00, 59.00it/s]


In [29]:
df[(df['ratio'] > 4) ]

,sentence,id,comment,score,ratio


In [30]:
class Aurora3:

    def __init__(self, df,sent_dic):
        self.df = df
        self.okt = konlpy.tag.Okt()
        self.sent_dic = sent_dic

    def get_df(self):# 최종 결과 반환
        print("문장을 토큰화 중입니다...")
        self.tokenizer_run()

        print("감성사전을 업데이트 중입니다...")
        self.expand_sent_dic()

        print("문장 감성분석 중입니다....")
        self.sent_analyze()
        return self.df

    def tokenizer_run(self): # 텍스트 전처리 & 토큰화
        tqdm.pandas()

        def text_preprocess(x):
            text=[]
            a = re.sub('[^가-힣0-9a-zA-Z\\s]', '',x)
            for j in a.split():
                text.append(j)
            return ' '.join(text)

        def tokenize(x):
            text = []
            tokens = self.okt.pos(x)
            for token in tokens :
                if token[1] == 'Adjective' or token[1]=='Adverb' or token[1] == 'Determiner' or token[1] == 'Noun' or token[1] == 'Verb' or 'Unknown':
                    text.append(token[0])
            return text
        self.df['comment'] = self.df['sentence'].apply(lambda x : text_preprocess(x))
        self.df['comment'] = self.df['comment'].progress_apply(lambda x: tokenize(x))

    def expand_sent_dic(self):
        sent_dic = self.sent_dic

        def make_sent_dict(x) :
            pos=[]
            neg=[]
            tmp={}

            for sentence in tqdm(x):
                for word in sentence :
                    target = sent_dic[sent_dic[0]==word]
                    if len(target)==1: # 기존에 있는 단어라면 그대로 사용
                        score = float(target[1])
                        if score > 0:
                            pos.append(word)
                        elif score < 0:
                            neg.append(word)
                    tmp[word] = {'W':0,'WP':0,'WN':0} # 감성사전 구성
            pos = list(set(pos))
            neg = list(set(neg))

            for sentence in tqdm(x):
                for word in sentence :
                    tmp[word]['W'] += 1 # 빈도 수
                    for po in pos :
                        if po in sentence:
                            tmp[word]['WP'] += 1 # 긍정단어과 같은 문장 내 단어일 때
                            break
                    for ne in neg:
                        if ne in sentence:
                            tmp[word]['WN'] += 1 # 부정단어와 같은 문장내 단어일 때
                            break
            return pos, neg, pd.DataFrame(tmp)

        def make_score_dict(d,p,n):
            N=sum(d.iloc[0,::])
            pos_cnt=sum(d.loc[::,p].iloc[0,::])
            neg_cnt=sum(d.loc[::,n].iloc[0,::])

            trans =d.T
            trans['neg_cnt']=neg_cnt
            trans['pos_cnt']=pos_cnt
            trans['N']=N

            trans['MI_P']=np.log2(trans['WP']*trans['N']/trans['W']*trans['pos_cnt'])
            trans['MI_N']=np.log2(trans['WN']*trans['N']/trans['W']*trans['neg_cnt'])
            trans['SO_MI']=trans['MI_P'] - trans['MI_N']

            trans = trans.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
            trans = trans.sort_values(by=['SO_MI'],ascending=False)
            return trans

        def update_dict(d):
            add_Dic = {0:[],1:[]}
            for i in d.T.items():
                if i[0] not in list(sent_dic[0]):
                    if len(i[0]) > 1:
                        add_Dic[0].append(i[0])
                        add_Dic[1].append(i[1]['SO_MI'])

            add_Dic=pd.DataFrame(add_Dic)
            Sentiment=pd.merge(sent_dic,add_Dic,'outer')
            return Sentiment

        self.pos, self.neg, self.new_dict = make_sent_dict(self.df['comment'].values)

        self.t_dict = make_score_dict(self.new_dict,self.pos,self.neg)
        self.t_dict['SO_MI'] = scaler.fit_transform(self.t_dict['SO_MI'].values.reshape(-1,1))

        self.add_dict =update_dict(self.t_dict)

    def sent_analyze(self): # 데이터 감성분석
        tqdm.pandas()

        def get_cnt(x):
            cnt = 0
            for word in list(set(x)):
                target = self.add_dict[self.add_dict[0]==word]
                if len(target)==1:
                    cnt += float(target[1])
            return cnt

        def get_ratio(x):
            score = x['score']
            length = np.log10(len(x['comment']))+1
            try:
                ratio= round(score/length,2)
            except:
                ratio = 0
            return ratio

        tqdm.pandas()
        self.df['score']= self.df['comment'].progress_apply(lambda x : get_cnt(x))
        self.df['ratio'] = self.df.apply(lambda x: get_ratio(x), axis = 1)

In [31]:
test = Aurora3(df,sent_dic)
res = test.get_df()

문장을 토큰화 중입니다...


100%|██████████| 20/20 [00:00<00:00, 174.88it/s]


감성사전을 업데이트 중입니다...


  0%|          | 0/20 [00:00<?, ?it/s]<ipython-input-30-cef4fa69e41d>:51: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  score = float(target[1])
100%|██████████| 20/20 [00:00<00:00, 19934.90it/s]
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.11/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log2
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-30-cef4fa69e41d>:100: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  Sentiment=pd.merge(sent_dic,add_Dic,'outer')


문장 감성분석 중입니다....


  0%|          | 0/20 [00:00<?, ?it/s]<ipython-input-30-cef4fa69e41d>:118: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  cnt += float(target[1])
100%|██████████| 20/20 [00:00<00:00, 62.59it/s]


In [32]:
res


,sentence,id,comment,score,ratio
0,감기에 걸려서 너무 피곤하고 아팠다.,0,"[감기, 에, 걸려서, 너무, 피곤하고, 아팠다]",-3.000000,-1.69
1,할 것들이 많아서 스트레스를 받았다.,1,"[할, 것, 들, 이, 많아서, 스트레스, 를, 받았다]",-2.000000,-1.05
2,주말을 너무 게으르게 보냈다. 작은거 하나라도 완성하는 의미있는 시간을 보내자!,2,"[주말, 을, 너무, 게으르게, 보냈다, 작은거, 하나, 라도, 완성, 하는, 의미...",-8.610739,-3.96
3,이해가 되지 않지만 그럴수도 있다고 생각해보자.,3,"[이해, 가, 되지, 않지만, 그럴수도, 있다고, 생각, 해보자]",0.000000,0.00
4,추가 근무가 들어와서 짜증이 났지만 도움이 될 수 있어서 좋다.,4,"[추가, 근무, 가, 들어와서, 짜증, 이, 났지만, 도움, 이, 될, 수, 있어서...",-1.652685,-0.78
5,서두르지 말고 힘을 빼자. 지치지 말고 바라보는 시간도 가져야 한다.,5,"[서두르지, 말고, 힘, 을, 빼자, 지치지, 말고, 바라보는, 시간, 도, 가져야...",0.231544,0.11
6,많이 지쳐서 잠깐만 쉴게요. 마음 속 상처를 돌봐줘야 해요.,6,"[많이, 지쳐서, 잠깐, 만, 쉴게요, 마음, 속, 상처, 를, 돌봐줘야, 해, 요]",-5.884228,-2.83
7,여러분들이 웃는 모습을 볼 수 있어서 좋아요. 여러분이 웃을 수 있다면 기꺼이 할 ...,7,"[여러분, 들, 이, 웃는, 모습, 을, 볼, 수, 있어서, 좋아요, 여러분, 이,...",2.884228,1.25
8,여전히 찝찝하고 편하지 않다. 누구에게 권할 자신이 없다.,8,"[여전히, 찝찝하고, 편하지, 않다, 누구, 에게, 권할, 자신, 이, 없다]",-1.000000,-0.50
9,누군가를 사랑하는 것은 어려운 것 같아요.,9,"[누군가, 를, 사랑, 하는, 것, 은, 어려운, 것, 같아요]",-2.652685,-1.36


In [34]:
res = res[['id', 'sentence', 'score', 'ratio']]
res.to_csv('my_senti_result.txt', sep='\t', index=False)